In [1]:
import json                                         
from pprint import pprint                   #pprint= preety print(prints complex data into readable format)
with open("data.json",mode="r") as f:       # if you open using 'with' then files is automatically closed (best practice)
    content=json.load(f)                #json.load = json-->python
                                        #json.dump = python-->json
print(content)

{'name': 'hari', 'age': '30', 'skills': 'web'}


In [2]:
aDict={
    "name":"hari",
    'age':'30',
    'skills':'web'
}
with open("data.json", mode="w") as f:
    json.dump(aDict, f,indent=4)        #indent means each nested level is intented(given) by 4 spaces 

In [3]:
import sqlite3
conn = sqlite3.connect("example.db")    #creates connection with the data base,if database doesnot exist, it creates a new one
cursor= conn.cursor()       #cursor is used to execute SQL queries

In [4]:
# create table
cursor.execute("""
        CREATE TABLE IF NOT EXISTS students(
               id INTEGER PRIMARY KEY,
               name TEXT,
               age INTEGER,
               address TEXT
            )
""")

In [5]:
# insert data
cursor.execute("INSERT INTO students(name,age,address) VALUES(?,?,?)",("Ram",34,"dharan"))
cursor.execute("INSERT INTO students(name,age,address) VALUES(?,?,?)",("hari",38,"biratnagar"))
cursor.execute("INSERT INTO students(name,age,address) VALUES(?,?,?)",("shyam",43,"ithari"))
conn.commit()

In [6]:
# query
cursor.execute("SELECT * FROM students")
rows = cursor.fetchall()
print(rows)

[(1, 'Ram', 34, 'dharan'), (2, 'hari', 38, 'biratnagar'), (3, 'shyam', 43, 'ithari')]


In [7]:
cursor.execute("SELECT * FROM students WHERE address= 'dharan' OR address= 'ithari'")
rows=cursor.fetchall()
for row in rows:
    print(row)

conn.close()

(1, 'Ram', 34, 'dharan')
(3, 'shyam', 43, 'ithari')


In [8]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///mydb.db', echo=True)

# Python → Engine → Database
# sqlite → Database type
# :/// → File-based database
# mydb.db → Database file name

#echo=True ; Displays all SQL statements executed by SQLAlchemy

In [9]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()                   # all python classes must inherit from Base
class Student(Base):                        # maps this class to a database table
    __tablename__='students'                #Specifies the table name in the database
    id=Column(Integer, primary_key=True)
    name=Column(String)
    age=Column(Integer)

# create the table
Base.metadata.create_all(engine)        #Uses the database connected by engine

2025-12-30 21:24:17,803 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-30 21:24:17,805 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2025-12-30 21:24:17,808 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-30 21:24:17,811 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2025-12-30 21:24:17,812 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-12-30 21:24:17,815 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	age INTEGER, 
	PRIMARY KEY (id)
)


2025-12-30 21:24:17,816 INFO sqlalchemy.engine.Engine [no key 0.00137s] ()
2025-12-30 21:24:17,826 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)         #Ensures sessions know which database to connect to
                                            #Engine → Session Factory → Session
session = Session()         #Creates an actual session object

In [11]:
# adding data 
student1 = Student(name='Ram', age=32)
student2 = Student(name='Hari', age=45)
session.add(student1)
session.add(student2)

session.commit()

2025-12-30 21:24:17,862 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-30 21:24:17,869 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age) VALUES (?, ?) RETURNING id
2025-12-30 21:24:17,870 INFO sqlalchemy.engine.Engine [generated in 0.00021s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Ram', 32)
2025-12-30 21:24:17,875 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age) VALUES (?, ?) RETURNING id
2025-12-30 21:24:17,877 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Hari', 45)
2025-12-30 21:24:17,879 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
# get all students
students = session.query(Student).all()
for s in students:
    print(s.name, s.age)

2025-12-30 21:24:17,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-12-30 21:24:17,901 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age 
FROM students
2025-12-30 21:24:17,904 INFO sqlalchemy.engine.Engine [generated in 0.00199s] ()
Ram 32
Hari 45


In [13]:
conn.close()
engine.dispose()